In [ ]:
#importing relevant modules
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
from matplotlib.animation import FFMpegWriter
from mpl_toolkits.basemap import Basemap
import os
import requests
import dateutil.parser
from datetime import timedelta, datetime
from obspy import UTCDateTime

In [ ]:
firstquakeid = "2014p051675" # user input Geonet public ID 

In [ ]:
quake = requests.get("https://api.geonet.org.nz/quake/2014p051675").json()
starttime = dateutil.parser.isoparse(quake['features'][0]['properties']['time'])
endtime = starttime + timedelta(days = 1)
start, end = starttime.strftime('%Y-%m-%dT%H:%M:%S'), endtime.strftime('%Y-%m-%dT%H:%M:%S')
lon, lat = quake['features'][0]['geometry']['coordinates'][0], quake['features'][0]['geometry']['coordinates'][1]
latlon = str(lon) + "+" + str(lat)

In [ ]:
quakejson = requests.get(
        "http://wfs.geonet.org.nz/geonet/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=geonet:quake_search_v1&outputFormat=json&cql_filter=origintime>=" 
        + start + "+AND+origintime<=" + end + "+AND+DWITHIN(origin_geom,Point+(" + latlon + "),50000,meters)").json()

In [ ]:
quakejson['features'][0]
lat, lon, mag, time = ([] for i in range(4))
for a in range(len(quakejson['features'])):
    lon.append(quakejson['features'][a]['geometry']['coordinates'][0])
    lat.append(quakejson['features'][a]['geometry']['coordinates'][1])
    mag.append(quakejson['features'][a]['properties']['magnitude'])
    time.append(UTCDateTime(quakejson['features'][a]['properties']['origintime']))
df = pd.DataFrame({"lat":lat, "lon":lon, "time":time,"mag":mag})
df = df.sort_values(by = "time")
df.head()

In [ ]:
fig, ax = plt.subplots(figsize = (15, 10), dpi = 100)
m = Basemap(projection = "merc", llcrnrlon = lon[0] - .25, llcrnrlat =  lat[0] - .25,
              urcrnrlon =  lon[0] + .25, urcrnrlat = lat[0] + .25, epsg = 2193)
m.arcgisimage(service = "World_Topo_Map", xpixels = 1500, ypixels = 1500)
timelist = [list(df['time'])[0] + 60 * x for x in range(300)]
stime = [str(UTCDateTime(start)+ 60 * i) for i in list(range(len(timelist)))]
roundmag = [(int(str(x)[0])+1)**4 * 2 for x in mag]
scat= m.scatter([],[], alpha = 0.5, edgecolor= "black", linewidths = 0.6)  
annotation = ax.annotate(str(stime[0]), xy=(400, 725), xycoords='axes pixels',  fontsize=10)
annotationcount = ax.annotate("Earthquake count: 0", xy = (20, 20), xycoords='axes pixels', fontsize = 10)
annotation.set_animated(True)
annotationcount.set_animated(True)
def init():
    scat.set_offsets(np.c_[0,0])
    return scat, annotation, annotationcount
def update(a):
    df2 = df[df['time'] < timelist[a]]
    x, y = m(list(df2['lon']), list(df2['lat']))
    scat.set_offsets(np.c_[x, y])
    scat._sizes=[(int(str(x)[0])+1)**2.5 * 2 for x in list(df2['mag'])]
    annotation.set_text(stime[a])
    annotationcount.set_text("Earthquake count: " + str(len(df2)))
    return scat,
ani = animation.FuncAnimation(fig, update, init_func=init, interval = 10, blit=False, frames = len(timelist))
writer = FFMpegWriter(fps=5, bitrate=1800)
ani.save(("banana.mp4"), writer=writer)